# National Coding Week Excercises - R!

## What is this?
This is a jupyter notebook. It is set up with all the data and software dependencies you need to have a go at coding in R/Python. 

## How do I use it?
You can run code by clicking in the code cell and pressing `ctrl`/`cmd` + `enter` or clicking the run button (below)

<img src="images/run-code.png" alt="image" style="height:150px;padding:10px;"/>

You can insert a new code 'chunk' by clicking the + button at the top left of the page (right image) 

<img src="images/insert-code-chunk-1.png" alt="image" style="height:150px;padding:10px;"/>

...and selecting code or markdown with the dropdown (left image).

<img src="images/code-markdown.png" alt="image" style="height:150px;padding:10px;"/>

## Before you start!

Make sure you set the language to R using the change kernel option.

<img src="images/set-kernel.png" alt="image" style="height:150px;padding:10px;"/>

# Coding Challenges:

Try coding in R by having a go at one of our challenges.

## Questions to answer:
* Which city has had the biggest temperature change?
* Which city has had the smallest temperature change?
* Can you make a plot of annual temperature changes for each city or region?
* Can you make a map of temperature changes?

## The Data

The dataset is a collection of daily temperature readings (daing back to 1995) from cities across the globe (split into 7 regions). It was [sourced from Kaggle](https://www.kaggle.com/sudalairajkumar/daily-temperature-of-major-cities) ([original source](https://academic.udayton.edu/kissock/http/Weather/default.htm)) and tidied up a bit to encode missing values and remove data before 995 (there were some readings for year 200 making it difficult to work with). 

The data is stored in the `data` folder. There is one csv of daily temperatures per region. A dataset of monthly mean temperatures for each city is also included in the `data` folder. This is easier to work with because it is much smaller than the daily temperature files. 

## Getting Started

If you are new to R and the tidyverse, have a read through [this cheatsheet](http://datacamp-community-prod.s3.amazonaws.com/c1fae72f-d2d7-4646-9dce-dd0f8fb5c5e8). It will give you the basics of processing and analysis.

# Basics

## Loading packages

R is built on packages which do different things. To use them you need to load them with `library(package_name)`. The following are already installed, load them using this code: 

In [8]:
library(readr)
library(dplyr)
library(tidyr)
library(ggplot2)
library(purrr)

## Loading the data
You can load the data using the `read_csv` function with the path to the file as the first argument. Don't forget to assign the result. 
E.g.

In [3]:
city_temps <- read_csv("data/monthly-mean-temps.csv")


-- Column specification --------------------------------------------------------
cols(
  Region = col_character(),
  Country = col_character(),
  State = col_logical(),
  City = col_character(),
  Year = col_double(),
  Month = col_double(),
  monthly_average = col_double()
)




To preview the data use the head function on the object you assigned the data to `city_temps`

In [4]:
head(city_temps)

Region,Country,State,City,Year,Month,monthly_average
<chr>,<chr>,<lgl>,<chr>,<dbl>,<dbl>,<dbl>
Africa,Algeria,NA,Algiers,1995,1,51.41935
Africa,Algeria,NA,Algiers,1995,2,54.80000
Africa,Algeria,NA,Algiers,1995,3,55.00000
Africa,Algeria,NA,Algiers,1995,4,57.19333
Africa,Algeria,NA,Algiers,1995,5,67.40968
Africa,Algeria,NA,Algiers,1995,6,71.72333


# Daily Temperatures

We can have a look at the daily temperatures data using the list files function (I'm displaying it as a tibble)

In [6]:
tibble(files = list.files("data", pattern = "daily", full.names = TRUE))

files
<chr>
data/daily-temperature-africa.csv
data/daily-temperature-asia.csv
data/daily-temperature-australia-south-pacific.csv
data/daily-temperature-europe.csv
data/daily-temperature-middle-east.csv
data/daily-temperature-north-america.csv
data/daily-temperature-south-central-america-carribean.csv
data/daily-temperature-us.csv


You can load all of the daily temperatures data into one data frame by reading each csv and binding them together:

In [10]:
files <- list.files("data", pattern = "daily", full.names = TRUE)
all_temps <- map(files, read_csv) %>% bind_rows()


-- Column specification --------------------------------------------------------
cols(
  Region = col_character(),
  Country = col_character(),
  State = col_logical(),
  City = col_character(),
  Month = col_double(),
  Day = col_double(),
  Year = col_double(),
  AvgTemperature = col_double()
)



-- Column specification --------------------------------------------------------
cols(
  Region = col_character(),
  Country = col_character(),
  State = col_logical(),
  City = col_character(),
  Month = col_double(),
  Day = col_double(),
  Year = col_double(),
  AvgTemperature = col_double()
)



-- Column specification --------------------------------------------------------
cols(
  Region = col_character(),
  Country = col_character(),
  State = col_logical(),
  City = col_character(),
  Month = col_double(),
  Day = col_double(),
  Year = col_double(),
  AvgTemperature = col_double()
)



-- Column specification --------------------------------------------------------
cols(
  Region 

Region,Country,State,City,Month,Day,Year,AvgTemperature
<chr>,<chr>,<lgl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Africa,Algeria,NA,Algiers,1,1,1995,64.2
Africa,Algeria,NA,Algiers,1,2,1995,49.4
Africa,Algeria,NA,Algiers,1,3,1995,48.8
Africa,Algeria,NA,Algiers,1,4,1995,46.4
Africa,Algeria,NA,Algiers,1,5,1995,47.9
Africa,Algeria,NA,Algiers,1,6,1995,48.7
Africa,Algeria,NA,Algiers,1,7,1995,48.9
Africa,Algeria,NA,Algiers,1,8,1995,49.1
Africa,Algeria,NA,Algiers,1,9,1995,49.0
